<h3 align="center"></h3>

<h1 align="center">Working Qwen 0.5b on GRPO</h1>

---

<h1 align="center">Training a Smol Math Reasoner with RL</h1>

This notebook is inspired by the [GRPO demo](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) by [will brown,](https://x.com/willccbb) training llama-1b on the gsm8k math dataset.

This will train a model, download the model, publish the model weights to your hugging face account, the run inference in python to your locally (almost) trained model returning inference to the terminal. It also saves down its reasoning along the way. This can be modified for almost any agent training so long as you define the reward functions and the data you want to train on. The idea is not to just train a model on a task but to show how little code is actually required to get a working model that is trained on a very specific task up and running in 2 hours. 


## Setting up the models.

## Understanding vLLM in this Project

### What is vLLM?
vLLM (Very Large Language Model) is a high-performance library developed by UC Berkeley's RISELab for efficient LLM inference and serving. It represents a significant advancement in LLM deployment technology, offering production-grade performance used by major companies like Databricks and Anyscale.

### Core Features and Benefits
1. **PagedAttention™ Technology**
   - Novel memory management system similar to operating system page management
   - Dramatically reduces memory usage during inference
   - Enables efficient handling of multiple requests simultaneously

2. **Performance Optimizations**
   - Continuous batching for dynamic request processing
   - Optimized CUDA kernels for maximum GPU utilization
   - Efficient KV cache management for transformer architectures
   - Supports both CPU and GPU inference

### Why vLLM is Critical for This Training Pipeline
1. **Speed Benefits**
   - Significantly faster inference during training
   - Essential for GRPO (Generative Reinforcement Policy Optimization)
   - Enables rapid model evaluation during reinforcement learning

2. **Memory Efficiency**
   - Allows both training and inference on the same GPU
   - Particularly important for our Qwen-0.5B model setup
   - Optimizes GPU memory usage through smart caching

### Installation Requirements
- Must be installed BEFORE TRL (Transformer Reinforcement Learning)
- Requires CUDA support for GPU acceleration
- Dependencies are automatically handled by pip

### Documentation & Resources
- Official Docs: [vllm.readthedocs.io](https://vllm.readthedocs.io/)
- GitHub: [github.com/vllm-project/vllm](https://github.com/vllm-project/vllm)
- Paper: ["vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention"](https://arxiv.org/abs/2309.06180)

### Important Note
After installing vLLM, you must restart the runtime before proceeding with other installations. This is due to a known interaction with the TRL library that requires vLLM to be installed first.

In [ ]:
%pip install vllm

# Understanding TRL and Hugging Face Datasets for Reinforcement Learning

This guide explains how two key tools—**TRL (Transformer Reinforcement Learning)** and the **Hugging Face Datasets library**—are used together to fine-tune language models with reinforcement learning (RL). The explanation is aimed at an informed undergraduate who is familiar with machine learning concepts.

---

## What Is TRL?

**TRL** is a specialized library built on top of Hugging Face's Transformers framework. Its purpose is to train language models using reinforcement learning techniques, where the model learns by receiving rewards rather than just imitating data.

### Key Components in Our Project

1. **GRPOConfig**
   - **Role:** A configuration class for GRPO (Generative Reinforcement Policy Optimization).
   - **Responsibilities:**
     - **Hyperparameter Management:** Sets learning rate, batch size, gradient accumulation steps, etc.
     - **Resource Allocation:** Manages GPU memory usage.
     - **Checkpointing:** Defines when and how often to save the model during training.
     - **Inference Settings:** Configures parameters for generating model outputs during evaluation.

2. **GRPOTrainer**
   - **Role:** The core engine that implements the RL training loop.
   - **Responsibilities:**
     - **Reward Integration:** Incorporates multiple reward functions to assess model outputs.
     - **Policy Optimization:** Updates model parameters based on computed rewards.
     - **Generation & Evaluation:** Produces text responses and evaluates their quality.
     - **Training State Management:** Keeps track of the progress and state of training.
     - **Efficient Inference:** Integrates with libraries like vLLM to speed up inference.

---

## What Is the Hugging Face Datasets Library?

The Hugging Face Datasets library is designed for efficient data handling. It simplifies the process of loading, processing, and streaming large datasets, which is particularly useful when working with language models.

### Usage in Our Project

1. **Data Loading**
   - We load datasets using a simple API. For example, to load the GSM8K (Grade School Math 8K) dataset:
     ```python
     from datasets import load_dataset, Dataset
     data = load_dataset('openai/gsm8k', 'main')
     ```
   - **Key Features:**
     - **Efficient Streaming & Caching:** Helps manage memory by streaming data.
     - **Data Versioning and Splitting:** Ensures reproducibility and proper separation of training/validation data.

2. **Data Processing**
   - Once loaded, the data is transformed to fit the needs of the training pipeline:
     - **Formatting:** Converts raw math problems into a prompt-friendly format.
     - **Instruction Integration:** Adds system-level instructions to guide the model.
     - **Structuring Data:** Creates input/output pairs that the model can learn from.
   - The `Dataset.map()` method is used to apply these transformations efficiently across the entire dataset.

---

## How TRL and Datasets Work Together

The integration of TRL and the Datasets library creates a smooth pipeline from raw data to model training:

1. **Data Flow**
   - **Step 1:** The Datasets library loads and processes the GSM8K dataset.
   - **Step 2:** The processed data is reformatted to include system prompts and user queries.
   - **Step 3:** This formatted data is then fed into the TRL training loop.

2. **Training Process**
   - **Reward Computation:** TRL computes rewards by evaluating how well the model’s responses match desired criteria.
   - **Policy Updates:** Based on these rewards, the model's parameters are updated to improve performance.
   - **Model Generation:** The model generates text, which is continually evaluated and refined.
   - **Optimization:** The overall training process is fine-tuned to enhance both learning efficiency and final performance.

---

## Why Use These Tools Together?

- **TRL's Strength:** Provides specialized reinforcement learning capabilities, which are essential for fine-tuning models based on complex reward signals.
- **Datasets' Efficiency:** Ensures that large-scale datasets are handled smoothly, from loading to processing.
- **Combined Benefit:** Together, they create a robust pipeline that:
  - Improves mathematical reasoning and problem-solving abilities.
  - Fine-tunes language models with more nuanced and targeted training.
  - Optimizes both the training workflow and the performance of the final model.

---

## Further Reading

For more detailed information, check out the official documentation:
- **TRL:** [github.com/huggingface/trl](https://github.com/huggingface/trl)
- **Datasets:** [huggingface.co/docs/datasets](https://huggingface.co/docs/datasets)


In [ ]:
%pip install trl datasets

## Defining the RL rewards


# Setting Up GRPO Training Components

This section details the setup required for GRPO training, covering the essential imports, response format definitions, and their roles in the overall training pipeline. The explanation below is designed for an informed undergraduate with a background in machine learning and programming.

---

## Core Imports and Their Purposes

The codebase leverages both basic Python libraries and specialized modules from Hugging Face to facilitate data handling, model management, and reinforcement learning.

### 1. Basic Python Libraries

- **`re` Module**  
  - **Purpose:** Implements regular expressions for pattern matching.  
  - **Usage:**  
    - Validates that the model's output adheres to a specified format.
    - Extracts specific answer segments from the overall model output.

- **`torch` Module**  
  - **Purpose:** PyTorch is used for deep learning and tensor computations.  
  - **Usage:**  
    - Handles low-level tensor operations critical for model computations.
    - Manages GPU acceleration, ensuring efficient training of large models.

### 2. Hugging Face Components

- **Datasets Library**  
  - **`load_dataset`:**  
    - **Purpose:** Loads datasets such as GSM8K, which contains grade-school math problems.
    - **Usage:** Simplifies data fetching and ensures data is readily available for training.
  - **`Dataset` Class:**  
    - **Purpose:** Provides a framework for managing and manipulating datasets.
    - **Usage:** Serves as the base for further data processing and transformation.

- **Transformers Library**  
  - **`AutoTokenizer`:**  
    - **Purpose:** Automatically handles tokenization of text inputs.
    - **Usage:** Converts raw text into tokenized sequences that the model can process.
  - **`AutoModelForCausalLM`:**  
    - **Purpose:** Loads pretrained causal language models.
    - **Usage:** Provides the base model that is fine-tuned using reinforcement learning techniques.

- **TRL Library**  
  - **`GRPOConfig`:**  
    - **Purpose:** Manages training configurations specific to the GRPO algorithm.
    - **Usage:** Sets key hyperparameters and resource configurations (e.g., learning rate, checkpointing).
  - **`GRPOTrainer`:**  
    - **Purpose:** Implements the GRPO training loop.
    - **Usage:** Orchestrates the reinforcement learning process including reward computation, policy updates, and efficient inference integration.

---

## Response Format Definition

To ensure the model outputs are both consistent and evaluable, a specific response format is defined.

### 1. System Prompt

The system prompt establishes the framework for how responses should be structured:
```
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
```
- **Purpose:**  
  - Enforces a clear separation between the reasoning (the thought process) and the final answer.
  - Ensures that the output is standardized, making it easier to evaluate the quality and correctness of both components.

### 2. XML Chain-of-Thought Format

- **Structure:**  
  - Utilizes a templated format where placeholders (e.g., `{reasoning}`, `{answer}`) are dynamically filled during model inference.
- **Benefits:**  
  - **Consistency:** Every response follows the same structure, ensuring uniformity.
  - **Ease of Parsing:** Facilitates automated extraction and evaluation of the reasoning and answer components.
  - **Logical Separation:** Clearly delineates the step-by-step reasoning process from the final numerical or factual answer.

---

## Purpose in Training Pipeline

The combination of these components underpins the efficiency and effectiveness of the training process:

- **Consistent Model Outputs:**  
  The defined response format ensures that the model consistently produces outputs in a predetermined, parseable structure.

- **Facilitates Reward Computation:**  
  A clear separation between reasoning and answer simplifies the process of assigning rewards based on specific aspects of the output.

- **Enables Clear Evaluation Metrics:**  
  With a standardized format, it becomes straightforward to measure the model's performance and the quality of its reasoning.

- **Supports Chain-of-Thought Reasoning:**  
  By enforcing a structured output, the training pipeline encourages the model to articulate its thought process, which can lead to improved transparency and reliability in decision making.

---

This setup not only standardizes the outputs but also streamlines the reinforcement learning training loop, ensuring that both the data processing and model training components work harmoniously for improved performance and evaluation.
```

First we set the general prompt structure (with the reasoning tags).

In [3]:
%pip install ipywidgets
%jupyter nbextension enable --py widgetsnbextension

  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
UsageError: Line magic function `%jupyter` not found.


In [1]:
import re                  # Regular expressions for pattern matching/text processing
import torch              # PyTorch for deep learning operations
from datasets import Dataset            # Just to annotate type usage, if needed
from transformers import AutoTokenizer, AutoModelForCausalLM  # Model loading
from trl import GRPOConfig, GRPOTrainer      # RL training components

# Load and prep dataset

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def format_openr1_math_example(example: dict) -> dict:
    """
    Given an example from OpenR1-Math-220k dataset, produce a 'prompt' + 'answer'
    structure that enforces <reasoning></reasoning><answer></answer> tags.

    Arguments:
        example (dict): A single record from the dataset that must include at least:
            - 'problem': the math problem text.
            - 'solution': a step-by-step chain-of-thought or partial reasoning (may or may not be fully correct).
            - 'answer': the final numeric or textual answer.

    Returns:
        dict: A dictionary with keys:
            - 'prompt': A list of role-based message objects, for system + user.
            - 'answer': The reference solution in the XML format.
    """
    # Extract relevant fields
    problem_text = example.get("problem", "")
    reasoning_text = example.get("solution", "")
    final_answer = example.get("answer", "")

    # Build the chat-style prompt
    prompt_list = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": problem_text}
    ]

    # Build the final text using the XML template
    xml_completion = XML_COT_FORMAT.format(
        reasoning=reasoning_text.strip(),
        answer=final_answer.strip()
    )

    return {
        "prompt": prompt_list,
        "answer": xml_completion
    }

# Example usage note:
# You would load the dataset externally:
#
# from datasets import load_dataset
# ds = load_dataset("open-r1/OpenR1-Math-220k", split="train")
#
# Then apply .map():
# ds = ds.map(format_openr1_math_example)
#
# Now each ds[i] has ds[i]["prompt"] and ds[i]["answer"] in the correct XML structure.
#
# You can feed ds[i]["prompt"] as input to your model, and ds[i]["answer"] as
# supervised label or reference for RL-based trainers.
#
# Done!


c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import trl.trainer.grpo_trainer because of the following error (look up to see its traceback):
No module named 'resource'

## Data Processing Functions for GSM8K Dataset

### Answer Extraction Functions

1. **XML Answer Extractor** (`extract_xml_answer`)
   - Purpose: Extracts answers from XML-formatted model outputs
   - Process:
     1. Splits text at `<answer>` tag
     2. Takes everything after the tag
     3. Splits at `</answer>` tag
     4. Takes everything before the closing tag
     5. Cleans whitespace
   - Used for: Processing model predictions during training

2. **Hash Answer Extractor** (`extract_hash_answer`)
   - Purpose: Extracts answers from GSM8K dataset format
   - Process:
     1. Checks for '####' delimiter
     2. Returns None if delimiter not found
     3. Takes everything after '####'
     4. Cleans whitespace
   - Used for: Processing ground truth answers from dataset

### Dataset Loading Function

`get_gsm8k_questions`
- Purpose: Prepares GSM8K dataset for GRPO training
- Parameters:
  - `split`: Dataset partition ('train' or 'test')
- Processing steps:
  1. Loads raw GSM8K data
  2. Transforms each example into training format:
     - Adds system prompt with format instructions
     - Includes user question
     - Extracts clean answer
- Output format:
  ```python
  {
      'prompt': [
          {'role': 'system', 'content': format_instructions},
          {'role': 'user', 'content': math_question}
      ],
      'answer': extracted_answer
  }
  ```

### Type Checking Notes
- Uses `# type: ignore` to suppress mypy warnings
- Maintains type hints for function signatures
- Ensures type safety where possible

In [ ]:
# Function to extract the answer from XML-formatted text
def extract_xml_answer(text: str) -> str:
    """
    Extracts the answer from text formatted with XML tags.

    Args:
        text (str): Input text containing <answer> tags

    Returns:
        str: The cleaned answer text between <answer> tags

    Example:
        Input: "<reasoning>some steps</reasoning><answer>42</answer>"
        Output: "42"
    """
    # Split on <answer> tag and take the last part (everything after the tag)
    answer = text.split("<answer>")[-1]
    # Split on </answer> tag and take the first part (everything before the closing tag)
    answer = answer.split("</answer>")[0]
    # Remove any leading/trailing whitespace and return
    return answer.strip()

# Function to extract answers from GSM8K dataset format
def extract_hash_answer(text: str) -> str | None:
    """
    Extracts the answer from GSM8K dataset format which uses #### as delimiter.

    Args:
        text (str): Input text containing #### delimiter

    Returns:
        str | None: The answer after #### or None if delimiter not found

    Example:
        Input: "Let's solve... #### 42"
        Output: "42"
    """
    # Check if the text contains the GSM8K answer delimiter
    if "####" not in text:
        return None
    # Split on #### and take everything after it, then clean whitespace
    return text.split("####")[1].strip()

# Function to load and format GSM8K dataset
def get_gsm8k_questions(split = "train") -> Dataset:
    """
    Loads and preprocesses the GSM8K dataset into a format suitable for GRPO training.

    Args:
        split (str): Dataset split to use ('train' or 'test')

    Returns:
        Dataset: Processed dataset with prompts and answers

    Note:
        # type: ignore comments are used to suppress mypy type checking warnings
    """
    # Load the GSM8K dataset from HuggingFace hub
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore

    # Transform each example in the dataset
    data = data.map(lambda x: { # type: ignore
        # Create a prompt list with system instruction and user question
        'prompt': [
            # System message containing format instructions
            {'role': 'system', 'content': SYSTEM_PROMPT},
            # User message containing the actual math question
            {'role': 'user', 'content': x['question']}
        ],
        # Extract and store the answer from the GSM8K format
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore

    return data # type: ignore

# Load the training dataset
dataset = get_gsm8k_questions()





# Understanding GRPO Reward Functions

## Overview of the Reward System
The training pipeline uses multiple reward functions to shape the model's behavior, each focusing on different aspects of the desired output. The total reward system can provide up to 3.5 points per response, carefully balanced across correctness and formatting criteria.

## Primary Reward Functions

### 1. Correctness Reward
- **Main Purpose**: Evaluates answer accuracy
- **Maximum Reward**: 2.0 points
- **Evaluation Process**:
  - Extracts model's answer from XML format
  - Compares with ground truth
  - Provides debugging output showing:
    - Original question
    - Expected answer
    - Full model response
    - Extracted answer
- **Scoring**: Binary reward (2.0 or 0.0)

### 2. Integer Format Reward
- **Main Purpose**: Ensures numerical responses
- **Maximum Reward**: 0.5 points
- **Evaluation Process**:
  - Checks if extracted answer is purely numerical
  - Validates digit-only responses
- **Importance**: Critical for mathematical problem-solving

## Formatting Reward Functions

### 3. Strict Format Verification
- **Maximum Reward**: 0.5 points
- **Requirements**:
  - Exact newline placement
  - Precise XML tag structure
  - Complete format compliance
- **Evaluation**: Uses rigid regular expression pattern
- **Purpose**: Maintains consistent response structure

### 4. Soft Format Verification
- **Maximum Reward**: 0.5 points
- **Flexibility**:
  - Allows variable whitespace
  - More forgiving tag placement
  - Maintains basic structure requirements
- **Purpose**: Backup formatting enforcement

## Detailed XML Structure Evaluation

### 5. XML Component Scoring
- **Maximum Total**: 0.5 points
- **Individual Components**:
  - Opening reasoning tag (0.125)
  - Closing reasoning tag (0.125)
  - Opening answer tag (0.125)
  - Closing answer tag (0.125)
- **Penalty System**:
  - Small deductions for excess text
  - Maintains cleanliness of response

### 6. Comprehensive XML Evaluation
- **Purpose**: Applies detailed scoring across all responses
- **Process**: Evaluates each response component
- **Importance**: Provides granular feedback

## Combined Impact on Training

### Total Reward Breakdown
1. Answer Correctness: 2.0 points
2. Numerical Format: 0.5 points
3. Strict Formatting: 0.5 points
4. Soft Formatting: 0.5 points
5. XML Structure: Up to 0.5 points

### Training Objectives
- **Primary Goal**: Correct mathematical reasoning
- **Secondary Goals**:
  - Clean, consistent formatting
  - Proper XML structure
  - Numerical answer provision
  - Clear solution presentation

### Behavioral Shaping
- Encourages step-by-step reasoning
- Promotes clear answer presentation
- Maintains consistent response structure
- Ensures numerical output format

This comprehensive reward system creates a balanced training signal that shapes the model's behavior across multiple dimensions, ensuring both accurate mathematical reasoning and clear, structured responses.


In [6]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]







# Deep Dive: GRPO Training Arguments Analysis

## Learning Rate (5e-6)
The learning rate of 5e-6 (0.000005) represents the step size in the gradient descent optimization process.

**Technical Details**:
- In standard SGD training of neural networks, learning rates often range from 1e-1 to 1e-3
- For LLM fine-tuning, we use much smaller rates (1e-5 to 1e-6) due to:
  1. Pre-trained model weights already encode complex patterns
  2. Large parameter count (500M in this case) means small changes propagate significantly
  3. Risk of "catastrophic forgetting" where new learning overwrites important pre-trained knowledge

**Research Basis**:
- Microsoft's paper on GPT-3 fine-tuning (2022) showed rates > 1e-5 led to instability
- Anthropic's research on constitutional AI used similar ranges (3e-6 to 8e-6)
- Meta's LLaMA fine-tuning guidelines recommend 5e-6 as a starting point

## Adam Optimizer Parameters
### Beta1 (0.9)
First moment estimate in Adam optimization.

**Technical Significance**:
- Controls exponential decay rate for momentum estimation
- 0.9 means each gradient update considers ~10 previous gradients
- Theoretical basis from Kingma & Ba's original Adam paper (2014)
- Higher values (>0.9) can:
  1. Lead to oscillation in loss landscape
  2. Miss fine-grained features in optimization space

### Beta2 (0.99)
Second moment estimate in Adam optimization.

**Technical Significance**:
- Controls variance estimation decay
- 0.99 provides longer-term memory of past gradients
- Research shows for LLMs:
  1. Lower values (<0.98) lead to training instability
  2. Higher values (>0.999) slow convergence significantly
  3. 0.99 balances stability and training speed



## Learning Rate Scheduler (Cosine)

**Smoothing the Rate:**
The cosine schedule smoothly transitions learning rates with the formula:
\[ lr \times 0.5 \times (1 + \cos(\pi \times \text{current\_step} / \text{total\_steps})) \]
This reduces oscillation and aids convergence.

**Benefits:**
- **Smooth Transition:** Avoids abrupt rate drops.
- **Initial Progress:** Faster early learning.
- **Better Convergence:** Fine-tunes weights effectively.

**Research Backing:**
DeepMind's Transformers saw better performance over step decay.

---

## Precision Settings (bfloat16)

**Efficient Computation:**
bfloat16 balances precision and speed. It offers a higher exponent range than FP16, aiding numerical stability.

### **Comparison:**
- **vs. FP32:** HALF the memory and faster.
- **vs. FP16:** MORE stability, LESS precision.

**Hardware Edge:**
Optimal for NVIDIA Ampere GPUs, reducing memory bandwidth needs.

---

## Batch Configuration

### **Per Device Train Batch Size (1)**

**Effective Use of Memory:**
Limits to 1 example per forward pass for memory efficiency and larger context windows.

### **Gradient Accumulation Steps (4)**

**Simulated Larger Batches:**
Accumulates over 4 steps to mimic a batch size of 4, enhancing stability.

**Research Note:**
Related to optimizer memory and batch size impact.

---

## Sequence Length Parameters

### **Max Prompt Length (256)**

**Balanced Context and Compute:**
 Limits to 256 tokens due to quadratic attention scaling. Suitable for most math problems.

### **Max Completion Length (200)**

**Reasoning Without Overstepping:**
Capped at 200 to allow detailed reasoning without excessive generation.

---

## Training Duration Parameters

### **Number of Training Epochs (1)**

**Quick Adaptation:**
One pass prevents overfitting and retains general skills, effective for RL policy adaptation.

**Research Context:**
Quick adaptation studied by Microsoft and Anthropic.

---

## Gradient and Memory Management

### **Max Gradient Norm (0.1)**

**Stable Updates:**
Caps gradients at 0.1 to prevent explosions, crucial for RL.

**Mathematical Impact:**
Ties to policy gradient variance and trust region methods.

---

## Hardware Utilization

### **vLLM Configuration**

**Efficient Memory Use:**
 Reserves 30% for KV cache and batch processing, optimizing throughput.

### **Device Specification (cuda:0)**

**Single GPU Optimized:**
Reduces communication overhead, maximizing bandwidth.

---

## Logging and Monitoring

**Simplified Setup:**
Disables logging to reduce I/O and dependency on external tools like Weights & Biases.

---

## Tokenizer Configuration

**Consistent Padding:**
Matches pad_token and eos_token for uniform behavior, aligning with HuggingFace practices.

---

This structured approach offers a deep understanding of each parameter, linking them to broader concepts and research contexts.







# Comprehensive Analysis of GRPO Training Parameters

## Weight Decay (0.1)
L2 regularization parameter controlling parameter magnitude.

**Technical Significance**:
- Higher than typical weight decay (usually 0.01-0.001) because:
  1. Helps prevent overfitting in low-data regime
  2. Maintains model's general capabilities while learning new tasks
  3. Acts as implicit early stopping mechanism

**Research Context**:
- Google's T5 paper showed higher weight decay (0.1) improved generalization
- OpenAI's fine-tuning studies indicate stronger regularization needed for instruction tuning
- Anthropic's research suggests correlation between weight decay and model calibration

## Warmup Ratio (0.1)
Fraction of total training steps used for learning rate warmup.

**Technical Details**:
- 10% of total steps use gradually increasing learning rate because:
  1. Prevents early training instability
  2. Allows model to adjust to new data distribution
  3. Particularly important with Adam optimizer due to early variance estimation

**Mathematical Basis**:
- Related to eigenspectrum of Hessian matrix
- Helps avoid poor early optimization trajectories
- Research shows correlation with batch size (larger batches need longer warmup)

## Learning Rate Scheduler (Cosine)
Controls learning rate decay pattern throughout training.

**Technical Implementation**:
- Follows cosine function: lr * 0.5 * (1 + cos(π * current_step / total_steps))
- Benefits over linear or step decay:
  1. Smooth transition between learning rates
  2. Faster initial progress
  3. Better final convergence properties

**Research Support**:
- DeepMind's Transformer papers show superior performance vs step decay
- Google Brain's extensive LR schedule comparisons
- Particularly effective with Adam optimizer in LLM context

## Precision Settings (bf16=True)
Uses Brain Float 16 format for computations.

**Technical Details**:
- Compared to FP16:
  1. Larger dynamic range (7 bits exponent vs 5)
  2. Lower precision mantissa (8 bits vs 10)
  3. Better numerical stability
- Compared to FP32:
  1. Half the memory usage
  2. Faster computation on modern GPUs
  3. Sufficient precision for LLM fine-tuning

**Hardware Considerations**:
- Optimal for NVIDIA Ampere architecture
- Reduces memory bandwidth requirements
- Enables larger effective batch sizes

## Batch Configuration
### Per Device Train Batch Size (1)
**Technical Rationale**:
- Single example per forward pass because:
  1. Maximizes available memory for model weights
  2. Reduces variance in gradient updates
  3. Allows larger context windows

### Gradient Accumulation Steps (4)
**Implementation Details**:
- Accumulates gradients over 4 forward passes because:
  1. Simulates larger batch size (effective batch size = 4)
  2. Reduces memory requirements
  3. Improves training stability

**Research Context**:
- Microsoft's DeepSpeed findings on gradient accumulation
- Relationship with optimizer state memory
- Impact on effective batch size calculations

## Generation Parameters
### Number of Generations (16)
**Technical Significance**:
- Multiple generations per prompt because:
  1. Enables exploration of response space
  2. Reduces variance in reward estimation
  3. Improves policy gradient estimation

**Statistical Basis**:
- Minimum samples needed for reliable policy gradient
- Trade-off between computation and estimation quality
- Impact on reward variance reduction


## Sequence Length Parameters

### Max Prompt Length (256)
**Technical Implementation**:
- Limits input sequence to 256 tokens because:
  1. Memory scales quadratically with sequence length (attention mechanism)
  2. Most math problems fit within this window
  3. Balances context window with computational efficiency

**Research Considerations**:
- Transformer attention complexity: O(n²)
- Token distribution analysis of GSM8K dataset
- Memory vs context window trade-offs

### Max Completion Length (200)
**Technical Rationale**:
- Caps generation length at 200 tokens because:
  1. Sufficient for step-by-step reasoning
  2. Prevents runaway generations
  3. Optimizes inference speed

**Empirical Basis**:
- Analysis of solution length distribution in GSM8K
- Memory requirements for beam search
- Impact on generation quality vs speed

## Training Duration Parameters

### Number of Training Epochs (1)
**Technical Significance**:
- Single pass through dataset because:
  1. Prevents overfitting on limited math examples
  2. Maintains general capabilities
  3. Sufficient for policy adaptation with RL

**Research Context**:
- Microsoft's findings on instruction fine-tuning
- OpenAI's studies on few-shot adaptation
- Anthropic's work on minimal fine-tuning

### Save Steps (100)
**Implementation Details**:
- Checkpoints every 100 steps because:
  1. Balances storage requirements
  2. Provides sufficient granularity for model selection
  3. Enables training recovery

**Practical Considerations**:
- Disk space requirements
- Checkpoint loading time
- Training resumption capabilities

## Gradient and Memory Management

### Max Gradient Norm (0.1)
**Technical Depth**:
- Clips gradient norm at 0.1 because:
  1. Prevents explosive gradients in RL setting
  2. Maintains stable policy updates
  3. Critical for convergence with policy gradients

**Mathematical Basis**:
- Relationship to policy gradient variance
- Impact on Wasserstein distance between policy updates
- Connection to trust region methods

## Hardware Utilization Parameters

### vLLM Configuration
**Technical Implementation**:
- GPU Memory Utilization (0.3 or 30%):
  1. Reserves memory for:
     - KV cache management
     - Dynamic batch processing
     - Continuous batching overhead
  2. Optimizes for:
     - PagedAttention mechanism
     - Inference throughput
     - Training stability

**Research Basis**:
- vLLM paper's memory analysis
- Empirical studies on GPU memory management
- Trade-offs between serving and training

### Device Specification (cuda:0)
**Technical Details**:
- Primary GPU designation because:
  1. Optimizes for single-GPU training
  2. Reduces communication overhead
  3. Maximizes memory bandwidth utilization

**Hardware Considerations**:
- PCIe bandwidth utilization
- CUDA stream management
- Memory transfer optimization

## Logging and Monitoring

### Log on Each Node (False)
**Technical Rationale**:
- Disables distributed logging because:
  1. Single-GPU setup
  2. Reduces I/O overhead
  3. Simplifies log analysis

### Reporting Configuration (none)
**Implementation Details**:
- Disables Weights & Biases because:
  1. Reduces network overhead
  2. Minimizes external dependencies
  3. Focuses on local performance analysis

**Practical Impact**:
- Reduced training overhead
- Simplified debugging
- Local-only experiment tracking

## Tokenizer Configuration
**Technical Significance**:
- Setting pad_token = eos_token because:
  1. Ensures consistent padding behavior
  2. Maintains model's probability distribution
  3. Critical for batch processing

**Research Context**:
- HuggingFace's tokenizer best practices
- Impact on attention mask computation
- Relationship to model architecture





In [ ]:
%pip install wandb

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: christian-cooper-us to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

output_dir="outputs/Qwen-0.5B-GRPO"
run_name="Qwen-0.5B-GRPO-gsm8k"

# Initialize wandb first
import wandb
wandb.init(
    project="qwen-math-grpo",
    entity="christian-cooper-us",
    config={
        "learning_rate": 5e-6,
        "adam_beta1": 0.9,
        "adam_beta2": 0.99,
        "weight_decay": 0.1,
        "warmup_ratio": 0.1,
        "batch_size": 1,
        "num_train_epochs": 1,
        "run_name": run_name
    }
)


training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=16,
    max_prompt_length=256,
    max_completion_length=200,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    log_on_each_node=False,
    use_vllm=True,
    vllm_gpu_memory_utilization=.3,
    vllm_device="cuda:0",
    report_to="wandb", #wandb is enabled.
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

And launch the actual training:

In [9]:
# use peft at your own risk; not working for me with multi-GPU training
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func],
    args=training_args,
    train_dataset=dataset,
    #peft_config=peft_config
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/trl/trainer/grpo_trainer.py:301: UserWarning: The requested device cuda:0 is also used for training. This may lead to unexpected behavior. It is recommended to use a dedicated device for vLLM.
  warnings.warn(


INFO 02-09 17:29:45 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 02-09 17:29:45 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen2.5-0.5B-Instruct, n

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 02-09 17:29:47 model_runner.py:1115] Loading model weights took 0.9279 GB
INFO 02-09 17:29:48 worker.py:267] Memory profiling takes 1.11 seconds
INFO 02-09 17:29:48 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.30) = 11.87GiB
INFO 02-09 17:29:48 worker.py:267] model weights take 0.93GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 9.41GiB.
INFO 02-09 17:29:49 executor_base.py:110] # CUDA blocks: 51373, # CPU blocks: 21845
INFO 02-09 17:29:49 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 25.08x
INFO 02-09 17:29:52 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:28<00:00,  1.23it/s]

INFO 02-09 17:30:20 model_runner.py:1562] Graph capturing finished in 28 secs, took 0.16 GiB
INFO 02-09 17:30:20 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 33.50 seconds


-------------------- Question:
Gerald wants to buy a meat pie that costs 2 pfennigs. Gerald has 54 farthings, and there are 6 farthings to a pfennig. How many pfennigs will Gerald have left after buying the pie? 
Answer:
7 
Response:
To solve this problem, we first need to understand how far possessions are worth in pfennigs and how far they are worth in farthings. We know that 5 farthings are equivalent to 1 pfennig. Therefore, in pfennigs, 54 farthings would be equivalent to:

\( 54 \text{ farthings} \times \frac{1 \text{ pfennig}}{5 \text{ farthings}} = 10.8 \text{ pfennigs} \)

Since Gerald has 10.8 pfennigs, after purchasing the meat pie, he will have:

\( 10.8 \text{ pfennigs} - 2 \text{ pfennigs} = 8.8 \text{ pfennigs} \)

So, Gerald will have 8.8 pfennigs left. 
Extracted:
To solve this problem, we first need to understand how far possessions are worth in pfennigs and how far they are worth in farthings. We know that 5 farthings are equivalent to 1 pfennig. Therefore, in pfenni

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


Streaming output truncated to the last 5000 lines.
 
Extracted:
200
-------------------- Question:
Jeanine bought 18 pencils and Clare bought half as many pencils. If Jeanine gave one third of his pencils to Abby, how many more pencils than Clare does Jeanine have now? 
Answer:
3 
Response:
<reasoning>
Jeanine bought 18 pencils, and Clare bought half as many, which is 18/2 = 9 pencils. Jeanine gave one third of his pencils to Abby, which is 18/3 = 6 pencils. After giving away 6 pencils, Jeanine has 18 - 6 = 12 pencils left. Now, Jeanine has 12 - 6 = 6 more pencils than Clare.
</reasoning>
<answer>
6
</answer>
 
Extracted:
6
-------------------- Question:
Josh found out that 7 bottle caps weigh exactly one ounce. Josh's entire bottle cap collection weighs 18 pounds exactly. How many bottle caps does Josh have in his collection? 
Answer:
2016 
Response:
<reasoning>
The weight of Josh's entire bottle cap collection is 18 pounds. Since 1 pound is equal to 16 ounces, we can calculate the to

TrainOutput(global_step=1868, training_loss=0.006439530964806228, metrics={'train_runtime': 6981.6392, 'train_samples_per_second': 1.07, 'train_steps_per_second': 0.268, 'total_flos': 0.0, 'train_loss': 0.006439530964806228})




# GRPO Training Execution Analysis

## Final Training Setup and Execution

### Core Components of the Training Pipeline

1. **Model Configuration**
```python
trainer = GRPOTrainer(
    model=model,                    # Qwen 0.5B model
    processing_class=tokenizer,     # Tokenizer for text processing
    reward_funcs=[...],            # Multiple reward functions
    args=training_args,            # Training configuration
    train_dataset=dataset          # GSM8K dataset
)
```

### Reward Function Order and Significance
The order of reward functions is crucial:
1. `xmlcount_reward_func`: Base format validation (0.5 max)
   - Provides granular feedback on XML structure
   - Acts as foundation for format learning

2. `soft_format_reward_func`: Lenient structure check (0.5 max)
   - Allows flexibility in formatting
   - Prevents over-penalization

3. `strict_format_reward_func`: Rigid format enforcement (0.5 max)
   - Ensures exact formatting compliance
   - Critical for consistent outputs

4. `int_reward_func`: Numerical validation (0.5 max)
   - Verifies numerical answers
   - Essential for mathematical accuracy

5. `correctness_reward_func`: Answer accuracy (2.0 max)
   - Primary learning signal
   - Highest reward weight

## Training Process Deep Dive

### What Actually Happens During Training

1. **Initialization Phase**
   - Model loaded into GPU memory
   - vLLM engine initialized (30% GPU memory)
   - Tokenizer prepared with padding configuration

2. **Per-Step Process**
   - Load math problem from GSM8K
   - Generate 16 different completions
   - Evaluate all reward functions
   - Compute policy gradient
   - Update model weights
   - Log progress and save checkpoints

3. **Observable Outputs**
   ```
   Question: [math problem]
   Answer: [expected]
   Response: [model output]
   Extracted: [processed answer]
   ```

### Training Duration and Resources
- Dataset: ~7,500 GSM8K problems
- Effective batch size: 4 (1 × 4 gradient accumulation)
- Total steps: ~1,875
- Expected runtime: 2-4 hours on A100
- Checkpoints: Every 100 steps

## Production Readiness Assessment

### Current Limitations

1. **Training Depth**
   - Single epoch may be insufficient
   - Limited exposure to problem variations
   - Potential underfitting

2. **Evaluation Gaps**
   - No validation set monitoring
   - Missing performance metrics
   - No systematic error analysis

3. **Infrastructure**
   - No deployment configuration
   - Missing monitoring setup
   - No model cards or documentation

### Required Steps for Production

1. **Model Validation**
   - Implement cross-validation
   - Create test suite
   - Perform behavioral testing
   - Safety assessment

2. **Performance Optimization**
   - Model compression
   - Inference optimization
   - Latency testing
   - Memory profiling

3. **Deployment Infrastructure**
   - Serving setup
   - Monitoring system
   - A/B testing framework
   - Rollback procedures

4. **Documentation Requirements**
   - Model cards
   - Usage guidelines
   - Performance characteristics
   - Known limitations

### PEFT Considerations
- Currently disabled due to multi-GPU issues
- Could enable:
  - Reduced memory footprint
  - Larger batch sizes
  - More efficient training
- Requires stability testing

## Recommendations for Production Deployment

1. **Extended Training Protocol**
   - Multiple epochs with validation
   - Early stopping implementation
   - Learning rate refinement
   - Batch size optimization

2. **Evaluation Framework**
   - Comprehensive test suite
   - Edge case analysis
   - Performance benchmarking
   - Safety evaluations

3. **Deployment Pipeline**
   - Model compression strategy
   - Serving infrastructure
   - Monitoring setup
   - Update procedures

4. **Documentation and Maintenance**
   - Detailed model cards
   - Regular updates
   - Performance monitoring
   - Incident response plan

This training setup provides a foundation but requires significant additional work for production deployment. It's currently more suitable for proof-of-concept or research purposes.













# Understanding Checkpoint-1868 (Final Checkpoint) Contents

## Core Model Files

### 1. Model Architecture Files
- `model.safetensors`: The actual model weights in safetensors format
  - More efficient than PyTorch's native format
  - Contains all neural network parameters
  - Weights, biases, embeddings, etc.

### 2. Configuration Files
- `config.json`: Model architecture configuration
  - Number of layers
  - Hidden dimensions
  - Attention heads
  - Model type/class
- `generation_config.json`: Default generation parameters
  - Max length
  - Temperature
  - Top-k/Top-p settings
  - Beam search config

### 3. Tokenizer Files
- `tokenizer.json`: Core tokenizer configuration
- `tokenizer_config.json`: Tokenizer settings
- `added_tokens.json`: Any custom tokens added
- `special_tokens_map.json`: Special token definitions
- `vocab.json`: The vocabulary file

### 4. Training State Files
- `trainer_state.json`: Training progress info
  - Steps completed
  - Loss history
  - Learning rates
- `training_args.bin`: Training configuration
- `optimizer.pt`: Optimizer state
- `scheduler.pt`: Learning rate scheduler state
- `rng_state.pth`: Random number generator state

## How Loading Works
When you call `from_pretrained()`, the process is:

1. **Architecture Detection**:
```python
model = AutoModelForCausalLM.from_pretrained(model_path)
```
- Reads `config.json` to determine model type
- Initializes correct model class
- Loads weights from `model.safetensors`

2. **Tokenizer Setup**:
```python
tokenizer = AutoTokenizer.from_pretrained(model_path)
```
- Reads `tokenizer_config.json`
- Loads vocabulary from `vocab.json`
- Configures special tokens

## Why This Works
- HuggingFace's transformers library expects this standard structure
- Each file has a specific purpose in model reconstruction
- AutoClasses detect and load appropriate model types
- All necessary information for inference is contained




Now run the final inference below that will call out to your model.

In [10]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# First, load the base model architecture
base_model = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# Load the fine-tuned weights
checkpoint_path = "outputs/Qwen-0.5B-GRPO/checkpoint-1868"  # Specific checkpoint folder
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    checkpoint_path,
    trust_remote_code=True
)

# Rest of the inference code remains the same
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def solve_math_problem(question: str):
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]

    input_text = tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.9,
        num_return_sequences=1,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Test with a sample problem
test_question = "A concrete factory makes cookies in batches of 124. If they need to make 1440 cookies for a large order and each batch takes 75 minutes to bake, how many hours will it take to complete the order if they can only bake one batch at a time?"
print("\nQuestion:", test_question)
print("\nResponse:", solve_math_problem(test_question))


Question: A concrete factory makes cookies in batches of 124. If they need to make 1440 cookies for a large order and each batch takes 75 minutes to bake, how many hours will it take to complete the order if they can only bake one batch at a time?

Response: system

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

user
A concrete factory makes cookies in batches of 124. If they need to make 1440 cookies for a large order and each batch takes 75 minutes to bake, how many hours will it take to complete the order if they can only bake one batch at a time?
assistant
<reasoning>
The number of batches required to make 1440 cookies is \( \frac{1440}{124} = 12.3068... \). Since each batch takes 75 minutes to bake, we convert this into hours by dividing by 60: \( \frac{75}{60} = 1.25 \) hours per batch. Therefore, for 12 batches (since you cannot bake more than one batch per time period), it will take \( 12 \times 1.25 = 15 \) hours.

However, since baking 

Save collab output to local zip

In [ ]:
%zip -r /content/model_checkpoint.zip /content/outputs/Qwen-0.5B-GRPO/checkpoint-1868

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copy to Drive
%cp -r /content/outputs/Qwen-0.5B-GRPO/checkpoint-1868 /content/drive/MyDrive/my_models/

push model weights to hugging face

In [11]:
%git config --global credential.helper store
%pip install huggingface-hub
%huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `CFAmodel` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/hugg

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load from your local checkpoint

tokenizer = AutoTokenizer.from_pretrained("/content/outputs/Qwen-0.5B-GRPO/checkpoint-1868")
model = AutoModelForCausalLM.from_pretrained("/content/outputs/Qwen-0.5B-GRPO/checkpoint-1868")

# Push to Hub (replace "username/your-model-name" with your repo name but this is how i set mine up)
model.push_to_hub("HarleyCooper/GRPOtuned2")
tokenizer.push_to_hub("HarleyCooper/GRPOtuned2")
